In [31]:
import math
import sys

import numpy as np

sys.path.append(f'../../FinanceHub')

from calendars import DayCounts
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np

dc = DayCounts('BUS/252', calendar='anbima')
today = pd.to_datetime('2021-04-26')

def forward_rate(i1, i2, n1, n2):
    t2 = (1 + i2)**(n2/252)
    t1 = (1 + i1)**(n1/252)
    return (t2/t1)**(252/(n2-n1)) - 1

In [32]:
contratos_di = pd.read_excel('DI_26042021.xlsx')
contratos_di["Vencimento BU"] = dc.following(pd.to_datetime(contratos_di["Data Vencimento"]))
contratos_di["DU"] = dc.days(today, contratos_di["Vencimento BU"])
contratos_di["Spot"] = (100000/contratos_di["Preço de ajuste Atual"])**(252/contratos_di["DU"]) - 1

In [54]:
contratos_di["Forward"] = np.zeros(len(contratos_di))

for i in range(3,12):
    contratos_di.loc[i,"Forward"] = forward_rate(
        contratos_di.loc[i-3]["Spot"],
        contratos_di.loc[i]["Spot"],
        contratos_di.loc[i-3]["DU"],
        contratos_di.loc[i]["DU"]
    )

for i in range(12, 29):
    contratos_di.loc[i,"Forward"] = forward_rate(
        contratos_di.loc[i-1]["Spot"],
        contratos_di.loc[i]["Spot"],
        contratos_di.loc[i-1]["DU"],
        contratos_di.loc[i]["DU"]
    )

contratos_di["Carry"] = contratos_di["Forward"] - contratos_di["Spot"]
print(contratos_di["Carry"].max())

carry_max = contratos_di["Carry"].idxmax()
print(contratos_di.loc[carry_max])

0.021727107362697007
Vencimento                                           V25
Preço de ajuste anterior                           71050
Preço de ajuste Atual                            71084.8
Variação                                           34.77
Valor do ajuste por contrato (R$)                  34.77
Data Vencimento                      2025-10-01 00:00:00
Vencimento BU                        2025-10-01 00:00:00
DU                                                  1117
Spot                                             0.08004
Forward                                         0.101767
Carry                                          0.0217271
DV01                                             34.0306
Carry_DV01                                   0.000638459
Name: 25, dtype: object


In [55]:
contratos_di["DV01"] = ((contratos_di["DU"]*(1+contratos_di["Spot"]))/252)*contratos_di["Preço de ajuste Atual"]*0.0001
contratos_di["Carry_DV01"] = contratos_di["Carry"]/contratos_di["DV01"]
dv01_max = contratos_di["Carry_DV01"].idxmax()
print(contratos_di.loc[dv01_max])

Vencimento                                           J22
Preço de ajuste anterior                         95446.4
Preço de ajuste Atual                            95446.7
Variação                                            0.28
Valor do ajuste por contrato (R$)                   0.28
Data Vencimento                      2022-04-01 00:00:00
Vencimento BU                        2022-04-01 00:00:00
DU                                                   237
Spot                                           0.0508001
Forward                                        0.0636067
Carry                                          0.0128066
DV01                                             9.43254
Carry_DV01                                    0.00135771
Name: 11, dtype: object


In [71]:
dv01_min = contratos_di[contratos_di["Carry_DV01"] > 0]["Carry_DV01"].idxmin()
print(contratos_di.iloc[dv01_min])
flattner_carry = contratos_di.loc[dv01_max]["Carry_DV01"] - contratos_di.iloc[dv01_min]["Carry_DV01"]
print("Flattner carry: ", flattner_carry*100, "%")


Vencimento                                           F26
Preço de ajuste anterior                         69594.2
Preço de ajuste Atual                            69638.5
Variação                                           44.34
Valor do ajuste por contrato (R$)                  44.34
Data Vencimento                      2026-01-01 00:00:00
Vencimento BU                        2026-01-02 00:00:00
DU                                                  1182
Spot                                              0.0802
Forward                                        0.0829531
Carry                                          0.0027531
DV01                                             35.2834
Carry_DV01                                   7.80282e-05
Name: 26, dtype: object
Flattner carry:  0.12796774363746521 %
